# Automatique ML Studio no-code

Ce module (*Automated ML*) proposé par Microsoft Azure Machine Learning Studio permet, à partir d'un jeu de données d'entrée, de rechercher le modèle et la configuration la plus optimale possible sans avoir besoin d'aucune connaissance particulière ! 

Après avoir obtenu le meilleur modèle (après 72 essaies de modèles différents) il est possible de déployer le modèle très facilement :

<br><br>

<img src="./img/automated-ml-deploy.jpg" style="margin-left:auto; margin-right:auto; display: block;">

<br><br>

L'API est ensuite exploitable directement.

In [1]:
import urllib.request
import json
import os

api_key = os.environ["ML_STUDIO_AUTOMATED_KEY"]

url = os.environ["ML_STUDIO_AUTOMATED_ENDPOINT"]
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

In [2]:
def prepare_body(list_value : list):
    data_send = [{"Column2":"","text":v} for v in list_value]
    data =  {
        "data": data_send,
        "method": "predict"
    }
    return str.encode(json.dumps(data))
    
def prepare_response(resp):
    json_result = json.loads(resp)
    values = eval(json_result)["result"]
    result = []
    for v in values:
        predict = v
        result.append(predict)
    return result

def send_request(list_value : list):
    result = None
    body = prepare_body(list_value)
    req = urllib.request.Request(url, body, headers) 
    try:
        response = urllib.request.urlopen(req)
        resp = response.read()

        result = prepare_response(resp)
            
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))

        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(json.loads(error.read().decode("utf8", 'ignore')))
    return result

In [3]:
res = send_request(["I hate banana!","I love banana!!"])

In [4]:
print(res)

['bad', 'good']


In [5]:
import pandas as pd
test_sample = pd.read_csv("./data/sample_test_dataset.csv",index_col=0)
print("Dimensionnalité :",test_sample.shape)
test_sample.head()

Dimensionnalité : (4000, 2)


,text,sentiment
0,Chillin @ the beach with my girl brit-brit wat...,good
1,@indraherlambang really wanna say that ure lik...,good
2,"@ambermac Gotta love Brad Pitt's performance, ...",good
3,Just realized that Matthew Sweet &amp; Susanna...,good
4,"@marksphone sorry, meant that to come from my ...",good


In [6]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,roc_auc_score
def benchmark(y_pred,y_true):
    print("accuracy score :",accuracy_score(y_true,y_pred))
    print(classification_report(y_true,y_pred))
    return pd.DataFrame(confusion_matrix(y_true,y_pred),columns=["good","bad"],index=["good","bad"])

In [7]:
X_test = test_sample["text"].tolist()
y_test = test_sample["sentiment"].tolist()

In [9]:
%%time
y_pred = send_request(X_test)

CPU times: user 11.6 ms, sys: 4.91 ms, total: 16.5 ms
Wall time: 1.76 s


In [12]:
benchmark(y_pred,y_test)

accuracy score : 0.76275
              precision    recall  f1-score   support

         bad       0.76      0.76      0.76      2000
        good       0.76      0.76      0.76      2000

    accuracy                           0.76      4000
   macro avg       0.76      0.76      0.76      4000
weighted avg       0.76      0.76      0.76      4000



,good,bad
good,1526,474
bad,475,1525


On obtient ici un score très acceptable pour un module no-code. Cette méthode permet de générer un modèle de classification en seulement 1H50 sans avoir besoin d'écrire une seule ligne de code et sans avoir besoin d'utiliser des modules drag & drop.
L'API est ensuite créée en seulement 1 clic et coûte très peu à l'utilisation (consomation 0.11€/h par utilisation CPU) soit ici un total de 0.22€ pour l'entraînement puis même pas 0,01€ pour effectuer le test !